In [1]:
from data_loader import load_data
from plot_utils import show_transform_plot
from functools import partial
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pickle as pkl
import os

In [2]:
train = load_data()

In [3]:
REPORTS_PATH = os.path.join(os.getcwd(), '..', 'reports')
print([f for f in os.listdir(REPORTS_PATH) if os.path.isfile(os.path.join(REPORTS_PATH, f))])

['coefs_ridge-2018-09-07', 'predicted_ridge-2018-09-07']


In [4]:
COEFS_FILE = 'coefs_ridge-2018-09-07'
COEFS_DIR = os.path.join(REPORTS_PATH, COEFS_FILE)
PREDICTED_FILE = 'predicted_ridge-2018-09-07'
PREDICTED_DIR = os.path.join(REPORTS_PATH, PREDICTED_FILE)
with open(COEFS_DIR, 'rb') as file:
    coefs = pkl.load(file)
with open(PREDICTED_DIR, 'rb') as file:
    predicted = pkl.load(file)

In [71]:
predicted = pd.Series(predicted, name='log_predicted')
failed_cond = lambda x: x>.4
diff = predicted-np.log(train['SalePrice'])
abs_diff = (abs(diff)).sort_values(ascending=False).reset_index(drop=False)
failed_idx = (abs_diff.where(failed_cond).dropna())['index'].tolist()
best_idx = (abs_diff.tail(len(failed_idx)))['index'].tolist()

failed = train.iloc[failed_idx]
best = train.iloc[best_idx]

In [60]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(failed)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1298,1299,60,RL,313.0,63887,Pave,NaN,IR3,Bnk,AllPub,Corner,Gtl,Edwards,Feedr,Norm,1Fam,2Story,10,5,2008,2008,Hip,ClyTile,Stucco,Stucco,Stone,796.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,5644,Unf,0,466,6110,GasA,Ex,Y,SBrkr,4692,950,0,5642,2,0,2,1,3,1,Ex,12,Typ,3,Gd,Attchd,2008.0,Fin,2,1418,TA,TA,Y,214,292,0,0,0,480,Gd,NaN,NaN,0,1,2008,New,Partial,160000
523,524,60,RL,130.0,40094,Pave,NaN,IR1,Bnk,AllPub,Inside,Gtl,Edwards,PosN,PosN,1Fam,2Story,10,5,2007,2008,Hip,CompShg,CemntBd,CmentBd,Stone,762.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,2260,Unf,0,878,3138,GasA,Ex,Y,SBrkr,3138,1538,0,4676,1,0,3,1,3,1,Ex,11,Typ,1,Gd,BuiltIn,2007.0,Fin,3,884,TA,TA,Y,208,406,0,0,0,0,NaN,NaN,NaN,0,10,2007,New,Partial,184750
632,633,20,RL,85.0,11900,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,7,5,1977,1977,Hip,CompShg,Plywood,Plywood,BrkFace,209.0,TA,Gd,CBlock,TA,TA,No,ALQ,822,Unf,0,564,1386,GasA,TA,Y,SBrkr,1411,0,0,1411,0,0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1977.0,Fin,2,544,TA,TA,Y,192,0,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Family,82500
1324,1325,20,RL,75.0,9986,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,BrkFace,428.0,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1795,1795,GasA,Ex,Y,SBrkr,1795,0,0,1795,0,0,2,0,2,1,Gd,7,Typ,1,Gd,Attchd,2007.0,RFn,3,895,TA,TA,Y,0,49,0,0,0,0,NaN,NaN,NaN,0,2,2007,New,Partial,147000
30,31,70,C (all),50.0,8500,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Feedr,Norm,1Fam,2Story,4,4,1920,1950,Gambrel,CompShg,BrkFace,BrkFace,None,0.0,TA,Fa,BrkTil,TA,TA,No,Unf,0,Unf,0,649,649,GasA,TA,N,SBrkr,649,668,0,1317,0,0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1920.0,Unf,1,250,TA,Fa,N,0,54,172,0,0,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,40000
462,463,20,RL,60.0,8281,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1965,1965,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,553,BLQ,311,0,864,GasA,Gd,Y,SBrkr,864,0,0,864,0,0,1,0,3,1,TA,5,Typ,1,Po,Detchd,1965.0,Unf,1,360,TA,TA,Y,0,0,236,0,0,0,NaN,GdWo,NaN,0,12,2009,WD,Normal,62383
968,969,50,RM,50.0,5925,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,3,6,1910,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,600,600,Grav,Fa,N,SBrkr,600,368,0,968,0,0,1,0,2,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,0,0,0,0,0,NaN,GdWo,NaN,0,5,2009,WD,Abnorml,37900
495,496,30,C (all),60.0,7879,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,5,1920,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,495,Unf,0,225,720,GasA,TA,N,FuseA,720,0,0,720,0,0,1,0,2,1,TA,4,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,523,115,0,0,0,NaN,GdWo,NaN,0,11,2009,WD,Abnorml,34900
1432,1433,30,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,4,6,1927,2007,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,656,656,GasA,TA,Y,SBrkr,968,0,0,968,0,0,2,0,4,1,TA,5,Typ,0,NaN,Detchd,1928.0,Unf,1,216,Fa,Fa,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,64500
970,971,50,RL,60.0,10800,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1.5Fin,4,4,1949,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,

In [59]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(best)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1189,1190,60,RL,60.0,7500,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1999,1999,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0,Unf,0,994,994,GasA,Gd,Y,SBrkr,1028,776,0,1804,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,1999.0,Fin,2,442,TA,TA,Y,140,60,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,189000
1209,1210,20,RL,85.0,10182,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Somerst,RRNn,Norm,1Fam,1Story,8,5,2006,2006,Hip,CompShg,VinylSd,VinylSd,Stone,420.0,Gd,TA,PConc,Ex,TA,Mn,GLQ,1220,Unf,0,440,1660,GasA,Ex,Y,SBrkr,1660,0,0,1660,1,0,2,0,3,1,Gd,8,Typ,1,Gd,Attchd,2006.0,RFn,2,500,TA,TA,Y,322,50,0,0,0,0,NaN,NaN,NaN,0,5,2006,New,Partial,290000
734,735,20,RL,NaN,8978,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1968,1968,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0,Unf,0,948,948,GasA,TA,Y,SBrkr,948,0,0,948,0,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1968.0,Unf,1,300,TA,TA,Y,147,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Family,108000
1346,1347,20,RL,NaN,20781,Pave,NaN,IR2,Lvl,AllPub,CulDSac,Gtl,NWAmes,PosN,Norm,1Fam,1Story,7,7,1968,2003,Hip,CompShg,BrkFace,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,297,Rec,68,1203,1568,GasA,TA,Y,SBrkr,2156,0,0,2156,0,0,2,0,3,1,TA,9,Typ,1,Gd,Attchd,1968.0,RFn,2,508,Gd,TA,Y,0,80,0,290,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,262500
1276,1277,60,RL,NaN,12936,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,6,1972,1972,Gable,CompShg,HdBoard,Plywood,None,0.0,TA,TA,CBlock,TA,Gd,No,BLQ,593,Unf,0,130,723,GasA,TA,Y,SBrkr,735,660,0,1395,0,1,1,1,3,1,TA,6,Typ,1,TA,Attchd,1972.0,Unf,2,497,TA,TA,Y,294,116,0,0,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal,162000
105,106,60,FV,75.0,9375,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,8,5,2003,2004,Hip,CompShg,VinylSd,VinylSd,BrkFace,768.0,Gd,TA,PConc,Ex,TA,No,Unf,0,Unf,0,1053,1053,GasA,Ex,Y,SBrkr,1053,939,0,1992,0,0,2,1,3,1,Gd,9,Typ,1,Gd,Attchd,2003.0,RFn,2,648,TA,TA,Y,140,45,0,0,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal,250000
862,863,20,RL,81.0,9672,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SawyerW,Norm,Norm,1Fam,1Story,6,5,1984,1985,Hip,CompShg,HdBoard,Plywood,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,338,Unf,0,702,1040,GasA,TA,Y,SBrkr,1097,0,0,1097,0,0,2,0,3,1,TA,6,Typ,0,NaN,Attchd,1986.0,Unf,2,480,TA,TA,Y,0,0,0,0,0,0,NaN,GdPrv,NaN,0,5,2010,WD,Normal,152000
145,146,160,RM,24.0,2522,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,Twnhs,2Story,6,5,2004,2006,Gable,CompShg,VinylSd,VinylSd,Stone,50.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,970,970,GasA,Ex,Y,SBrkr,970,739,0,1709,0,0,2,0,3,1,Gd,7,Maj1,0,NaN,Detchd,2004.0,Unf,2,380,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal,130000
837,838,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrDale,Norm,Norm,Twnhs,2Story,6,5,1973,1973,Gable,CompShg,HdBoard,HdBoard,BrkFace,158.0,TA,TA,CBlock,TA,TA,No,BLQ,330,Unf,0,153,483,GasA,TA,Y,SBrkr,483,504,0,987,1,0,1,1,2,1,TA,5,Typ,0,NaN,Detchd,1973.0,Unf,1,264,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2008,WD,Normal,100000
845,846,85,RL,NaN,16647,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,SFoyer,5,5,1975,1981,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,1390,Unf,0,0,1

In [108]:
(abs(diff[train['BsmtFinType2']!='Unf'])).sort_values(ascending=False)

462     0.569734
313     0.229512
107     0.200910
772     0.184357
17      0.179814
808     0.170076
322     0.166262
1048    0.159123
666     0.153981
52      0.150822
944     0.150461
330     0.141520
440     0.140419
520     0.140068
1049    0.137254
1220    0.136225
116     0.136065
271     0.133603
1215    0.133113
680     0.130792
1067    0.130465
129     0.129630
512     0.126939
223     0.123799
1216    0.123650
371     0.123456
992     0.120628
940     0.120280
273     0.116829
548     0.115145
          ...   
543     0.011576
352     0.011553
695     0.010156
166     0.010087
43      0.009887
778     0.009317
580     0.009112
661     0.008848
633     0.007083
182     0.006962
174     0.006825
1205    0.006788
1232    0.006473
177     0.006442
745     0.005555
39      0.004790
1396    0.004737
1024    0.004374
90      0.003799
1456    0.003669
1418    0.003623
470     0.003489
663     0.003439
1045    0.003344
894     0.002967
1308    0.002425
1179    0.002122
263     0.0018

#### Observations
TM = Two most failed
1. Most of failed are overpriced by model
- TM bad predicted has abnormal LotFrontage wich is placed far in tail of the LotFrontage distribution; note that this LotFrontage has been dropped and this 2 examples are quite far away with error from the next
- Also TM has irregular LotShape, however this irregularities occurs also in the best exmaples
- TM has LandContour Bnk, however many examples with the best loss hass also Bnk
- There are many different SaleConditions in failed
- In TM 2ndFlrSF is 2 times and more smaller tha 1stFlrSF (this model has indicator appllied to 2ndFlrSF)

#### Conclusions
1. Two examples with biggest loss strongly correlated with LotForntage 

### Ideas

- add OverallQual to OverallCond ratio
- replace 2ndFlrSF indicated to ratio of 2ndFlrSF to 1stFlrSF

In [94]:
abs_coefs = coefs
abs_coefs['weight'] = abs(coefs['weight'])
(abs_coefs).sort_values(by='weight', ascending=False).reset_index()

,index,name,weight
0,199,MSZoning_C (all),0.141032
1,9,GrLivArea,0.126136
2,241,RoofMatl_ClyTile,0.103238
3,235,Neighborhood_StoneBr,0.087740
4,219,Neighborhood_Crawfor,0.086632
5,220,Neighborhood_Edwards,0.082454
6,142,Functional_Typ,0.078088
7,17,OverallQual,0.067119
8,229,Neighborhood_NridgHt,0.066994
9,137,Functional_Maj2,0.066657


In [123]:
l = [4] *2


TypeError: unsupported operand type(s) for /: 'int' and 'list'